<a href="https://colab.research.google.com/github/King-of-Haskul/Machine-Learning-And-Data-Science/blob/main/Style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Neurat style transfer
General process:
1. Set up a network that computes VGG19 layer activations for the style-refernce image, the target image, and the generated image at the same time.

2. Use the layer activations computed over these three images to define the loss function described earlier, which you'll minimize in order to achieve style transfer.

3. Set up a gradient-descent process to minimize this loss function.

In [72]:
from google.colab import files
uploaded = files.upload()

Saving pic_2.jpg to pic_2.jpg


In [73]:
#Defining initial variables
from tensorflow.keras.preprocessing.image import load_img, img_to_array

target_image_path = '/content/pic_2.jpg'  #Path to the image we want to transform
style_reference_image_path = '/content/abstract.jpg' #Path to the style image

width, height = load_img(target_image_path).size
img_height = 400  #Dimensions of the generated image
img_width = int(width * img_height / height)

In [74]:
#Auxiliary functions
import numpy as np
from tensorflow.keras.applications import vgg19

def preprocess_image(image_path):
  img = load_img(image_path, target_size=(img_height, img_width))
  img = img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = vgg19.preprocess_input(img)
  return img

def deprocess_image(x):
  #Zero-centering by removing the mean pixel value from ImageNet. This reverses a transformation done by vgg19.preprocess_input
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1] #Converts images from 'BGR' to 'RGB', again another reversal of vgg19.preprocess_input
  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [75]:
#Loading the pretrained VGG19 network and applying it to the tree images
from tensorflow.keras import backend as K
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))  #Placeholder that will contain the generated image

input_tensor = K.concatenate([target_image, style_reference_image, combination_image], axis=0)

model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)
print('Model loaded.')

Model loaded.


In [76]:
#Content Loss: It will make sure the top layer of the VGG19 convnethas a similar view of the target image and the generated image
def content_loss(base, combination):
  return K.sum(K.square(combination - base))

In [77]:
#Style Loss: Uses an auxiliary function to compute the Gram matrix of an input matrxis i.e. a map of correlations found in the original feature matrix
def gram_matrix(x):
  features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
  gram = K.dot(features, K.transpose(features)) #inner product of features
  return gram

def style_loss(style, combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  channels = 3
  size = img_height * img_width
  return K.sum(K.square(S - C)) / (4. * (channels**2) * (size ** 2))

In [78]:
#Total Variation loss: It encourages spatial continuity in the generated image, thus avoiding overly pixelated results
def total_variation_loss(x):
  a = K.square(
      x[: , : img_height - 1, : img_width - 1, :] - x[:, 1:, :img_width - 1, :]
  )
  b = K.square(
      x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, :img_width - 1, :]
  )
  return K.sum(K.pow(a + b, 1.25))

In [79]:
#Defining the final loss that we have to minimize, which is a weighted average of the above three loses

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'  #Layer used for content loss
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1'] #Layers used for style loss

total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025 #Higher content-weight means the target content will be more recognizable in the generated image

#Adding the content loss
loss = K.variable(0.) #Defining loss by adding all component to this scalar variable
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + (content_weight * content_loss(target_image_features, combination_features))

#Adding the syle loss component for each target layer
for layer_name in style_layers:
  layer_features = outputs_dict[layer_name]
  style_reference_features = layer_features[1, :, :, :]
  combination_features = layer_features[2, :, :, :]
  sl = style_loss(style_reference_features, combination_features)
  loss = loss + ((style_weight / len(style_layers))* sl)

#Adding the total variational loss
loss = loss + total_variation_weight*total_variation_loss(combination_image)

In [80]:
combination_image

<tf.Tensor 'Placeholder_231:0' shape=(1, 400, 301, 3) dtype=float32>

In [81]:
#Setting up the gradient-descent process
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

grads = K.gradients(loss, combination_image)[0] #Gets the gradients of the generated image with regard to the loss

fetch_loss_and_grads = K.function([combination_image], [loss, grads]) #Function to fetch the values of the current loss and the current gradients

class Evaluator(object): 
  #This class wraps fetch_loss_and_grads in a way that lets you retrieve the losses and gradients via two seperate method calls, which is required by SciPy optimizer
  def __init__(self):
    self.loss_value = None
    self.grads_value = None
  
  def loss(self, x):
    assert self.loss_value is None
    x = x.reshape((1, img_height, img_width, 3))
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    self.loss_value = loss_value
    self.grad_values = grad_values #stored as cache
    return self.loss_value
  
  def grads(self, x):
    assert self.loss_value is not None
    grad_values = np.copy(self.grad_values)
    self.loss_value = None
    self.grad_values = None #Resetting the object
    return grad_values
  
evaluator = Evaluator()

In [84]:
#Style-transfer loop using L-BFGS algorithm for optimization

from scipy.optimize import fmin_l_bfgs_b
from imageio import imwrite
import time

result_prefix = 'my_result'
iterations = 20

x = preprocess_image(target_image_path)
x = x.flatten()

for i in range(iterations):
  print('Start of iteration', i)
  start_time = time.time()
  x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, evaluator.grads, maxfun=20)
  #Above line runs L-BFGS optimizations over the pixels of the generated image to minimze the neural style loss
  print('Current loss value:', min_val)
  img = x.copy().reshape((img_height, img_width, 3))
  img = deprocess_image(img)
  fname = result_prefix + 'at_iteration_%d.png' % i
  imwrite(fname, img)
  print('Image saved as ', fname)
  end_time = time.time()
  print('Iteration %d completed in %ds' % (i, end_time - start_time)


"\nfor i in range(iterations):\n  print('Start of iteration', i)\n  start_time = time.time()\n  x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, evaluator.grads, maxfun=20)\n  #Above line runs L-BFGS optimizations over the pixels of the generated image to minimze the neural style loss\n  print('Current loss value:', min_val)\n  img = x.copy().reshape((img_height, img_width, 3))\n  img = deprocess_image(img)\n  fname = result_prefix + 'at_iteration_%d.png' % i\n  imwrite(fname, img)\n  print('Image saved as ', fname)\n  end_time = time.time()\n  print('Iteration %d completed in %ds' % (i, end_time - start_time)\n"